In [1]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

In [3]:
import keras
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import *
from keras.optimizers import Adam

In [4]:
np.set_printoptions(suppress=True)

# MULTIPLE ITEMS

## Generate data

In [5]:
n = 5000

In [6]:
prices = np.random.uniform(50, 200, size=(n, 1)).astype(np.float32)
xs = np.random.normal(size=(n,5)).astype(np.float32)

In [7]:
# create input for item ids, we'll make 5
ids = np.tile([0,1,2,3,4], n//5).reshape(-1,1)

In [8]:
# create beta coefficients
beta_0 = np.array([500],)

beta_p = np.array([-1.4,
                   -1.5,
                   -1.6,
                   -1.8,
                   -2.0],)

betas_x = np.random.normal(size=(5,1)).astype(np.float32)
# betas_x

In [9]:
# optimal prices
opt_prices = (-beta_0/(2*beta_p)).round(2)
opt_prices

array([178.57, 166.67, 156.25, 138.89, 125.  ])

In [10]:
# noise
noise = np.random.normal(size=(n,1)).astype(np.float32)

In [11]:
# create demand to forecast
y = beta_0 + beta_p[ids]*prices + np.dot(xs, betas_x) + noise
y = np.maximum(0, y)

In [12]:
# plt.hist(y)

In [13]:
trn = np.arange(int(n*.8))
tst = np.arange(int(n*.8), n)

## Model demand

In [14]:
# price and X inputs
inp_p = Input((1,), name='inp_p')
inp_x = Input((5,), name='inp_x')

In [15]:
# going to use embeddings to learn the "coefficient" for each item
inp_ids = Input((1,), name='inp_ids')
ids_emb = Embedding(input_dim=5, output_dim=1, name='ids_emb')(inp_ids)
ids_emb = Reshape((1,), name='ids_emb_re')(ids_emb)

# multiply the embedding coefficient by price
x = Multiply(name='mult_p_emb')([ids_emb, inp_p])

In [16]:
# concat the inputs
x = Concatenate(name='concat')([x, inp_x])

# single dense hidden layer
x = Dense(10, name='hid')(x)

# output for predicting demand
out = Dense(1, name='out')(x)

# build and compile the model
model = Model([inp_ids, inp_p, inp_x], out)
model.compile(Adam(lr=.01), loss='mean_squared_error')

In [17]:
# train
hist = model.fit([ids[trn], prices[trn], xs[trn]], y[trn],
                 validation_data=[[ids[tst], prices[tst], xs[tst]], y[tst]],
                 epochs=50)

Train on 4000 samples, validate on 1000 samples
Epoch 1/50
4000/4000 [==============================] - 0s 89us/step - loss: 39606.5373 - val_loss: 27731.9032
Epoch 2/50
4000/4000 [==============================] - 0s 74us/step - loss: 26204.2215 - val_loss: 27040.9711
Epoch 3/50
4000/4000 [==============================] - 0s 70us/step - loss: 25456.9604 - val_loss: 26112.5016
Epoch 4/50
4000/4000 [==============================] - 0s 78us/step - loss: 24253.8289 - val_loss: 24425.5273
Epoch 5/50
4000/4000 [==============================] - 0s 77us/step - loss: 22018.0443 - val_loss: 21016.6730
Epoch 6/50
4000/4000 [==============================] - 0s 79us/step - loss: 17616.8102 - val_loss: 15142.1739
Epoch 7/50
4000/4000 [==============================] - 0s 78us/step - loss: 11296.0668 - val_loss: 9083.6958
Epoch 8/50
4000/4000 [==============================] - 0s 76us/step - loss: 6878.3770 - val_loss: 6292.6108
Epoch 9/50
4000/4000 [==============================] - 0s 74us/ste

## Price optimization

In [18]:
# make model copy for price
model_p = keras.models.clone_model(model)
model_p.set_weights(model.get_weights())

In [19]:
# "freeze" the model
for layer in model_p.layers:
    layer.trainable = False

In [20]:
ls = {l.name:l for l in model_p.layers}
ls

{'concat': <keras.layers.merge.Concatenate at 0x10fd1a0f0>,
 'hid': <keras.layers.core.Dense at 0x11dc08860>,
 'ids_emb': <keras.layers.embeddings.Embedding at 0x11dbef4e0>,
 'ids_emb_re': <keras.layers.core.Reshape at 0x10fd84f60>,
 'inp_ids': <keras.engine.topology.InputLayer at 0x10a46ba20>,
 'inp_p': <keras.engine.topology.InputLayer at 0x10a46b470>,
 'inp_x': <keras.engine.topology.InputLayer at 0x10fd1a048>,
 'mult_p_emb': <keras.layers.merge.Multiply at 0x10fd1ac18>,
 'out': <keras.layers.core.Dense at 0x11dc1cf98>}

In [21]:
# id price embeddings
# lets us "train" a price for every id
p = Embedding(input_dim=5, output_dim=1, name='ids_price')(ls['inp_ids'].output)
p = Reshape((1,), name='ids_price_re')(p)

In [22]:
x = ls['mult_p_emb']([p, ls['ids_emb_re'].output, ls['inp_p'].output])

In [23]:
# concat the inputs and send through hidden layers like before
x = ls['concat']([x, ls['inp_x'].output])
x = ls['hid'](x)
x = ls['out'](x)

In [24]:
# our output of the last model was an estimate for demand
# so now we multiply that by the price to get revenue
out = Multiply(name='revenue_out')([p, x])

In [25]:
# build model
model_p = Model([ls['inp_ids'].input, ls['inp_p'].input, ls['inp_x'].input], out)

In [26]:
# revenue loss
def rev_loss(y_true, y_pred):
    return -K.mean(y_pred)

In [27]:
# compile model
model_p.compile(Adam(lr=.1), loss=rev_loss)

In [28]:
# callback to track layer weights
class WtTracker(keras.callbacks.Callback):
    def __init__(self, layer, display=5):
        self.seen = 0
        self.layer = layer
        self.display = display

    def on_epoch_end(self,batch,logs={}):
        self.seen += 1
        if self.seen % self.display == 0:
            wts = self.layer.get_weights()[0].flatten().round(2)
            print(f'epoch {self.seen}, price {wts}')

In [29]:
# calculated optimal prices for reference
list(opt_prices)

[178.57, 166.67, 156.25, 138.89, 125.0]

In [30]:
# train
hist = model_p.fit([ids, np.ones((len(xs), 1)), xs], y,
                   epochs=50,
                   verbose=0,
                   callbacks=[WtTracker(model_p.get_layer('ids_price'))])

epoch 5, price [66.04 65.89 66.4  64.46 64.04]
epoch 10, price [117.38 115.48 113.8  107.89 103.51]
epoch 15, price [152.25 147.4  141.93 130.25 120.61]
epoch 20, price [170.64 162.62 153.57 137.19 124.5 ]
epoch 25, price [176.88 166.97 156.14 138.17 124.84]
epoch 30, price [177.93 167.52 156.37 138.22 124.85]
epoch 35, price [177.99 167.54 156.37 138.21 124.85]
epoch 40, price [177.99 167.54 156.38 138.21 124.83]
epoch 45, price [177.99 167.55 156.35 138.21 124.87]
epoch 50, price [177.99 167.57 156.4  138.17 124.84]
